## Final Project Submission

Please fill out:
* Student name: JOHN KIOKO
* Student pace: full time
* Scheduled project review date/time: Fri, Sep 15, 2023
* Instructor name: Lucille Kahela
* Blog post URL:


## Introduction

#### a) Business Problem

>Microsoft sees all the big companies creating original video content and they want to get in on the fun. They have decided to create a new movie studio, but they don’t know anything about creating movies. You are charged with exploring what types of films are currently doing the best at the box office.

>Problem Statement: You must then translate those findings into actionable insights that the head of Microsoft's new movie studio can use to help decide what type of films to create.

#### b) Main Objective
Finding the types of films that are doing best at the box office and use the data to create a new movie studio

#### c) Other Objectives

* To determine the relationships between genres and various predictor variables ( using gross, rating and the number of votes)
* Determine the relationships between the Movie_titles and predicting values (like rating, gross, and number of votes)
* Determin how average rating and number of votes will affect movies to be produced

## Data Understanding

## Loading Libraries, Datasets and Understanding Them

### Loading Libraries

In [1]:
# Your code here - remember to use markdown cells for comments as well!
# importing necessary libraries
import pandas as pd
import numpy as np
import csv
import shutil
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

#Establishing a connection (Sqlite Dataset)
import sqlite3
conn = sqlite3.connect('im.db')

import warnings
warnings.filterwarnings('ignore')

### Loading DATASETS

In [2]:
# Loading the datasets

bom_movie_gross = pd.read_csv('bom.movie_gross.csv')
rt_movie_info = pd.read_csv('rt.movie_info.tsv', sep='\t')
rt_reviews = pd.read_csv('rt.reviews.tsv', sep='\t', encoding= 'latin1')
tmdb_movies = pd.read_csv('tmdb.movies.csv')
movie_budgets = pd.read_csv('tn.movie_budgets.csv')

### Cleaning Datasets and Previewing them (Data UNDERSTANDING)

In [3]:
# checking the movie budgets dataset
bom_movie_gross

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010
...,...,...,...,...,...
3382,The Quake,Magn.,6200.0,NaN,2018
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,El Pacto,Sony,2500.0,NaN,2018
3385,The Swan,Synergetic,2400.0,NaN,2018


The foreign gross has a few nan values 

In [4]:
# Further data preview (Box office mojo dataset)
bom_movie_gross.shape

(3387, 5)

In [5]:
bom_movie_gross.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB
